<a href="https://colab.research.google.com/github/shubhamcodez/Data-science-tools/blob/main/image_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation in NumPy

You can use the following notebook to create new images using Data Augmentation.
Note: You can use functions seperately with complete control over all transformation factors, 
mixing the function implementations rather than linearly using it may give better results 

https://colab.research.google.com/drive/1a-U-odzlPeTIToPi3IwyFyxjz6d367bM?usp=sharing

In [44]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.ndimage import rotate
sns.set(color_codes=True)
from skimage.transform import resize, rescale

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import glob
from PIL import Image
import PIL
image_limit = 2500 # change according to ram availiblity
image_path = '/content/drive/MyDrive/train/i/'
destination_path = '/content/drive/MyDrive/train/i/'
image_collection = glob.glob(image_path + '*.png')
size = 256
images = np.array([np.array(Image.open(img).convert('RGB').resize((size, size), Image.ANTIALIAS)) for img in image_collection])

In [47]:
images.shape

(1, 256, 256, 3)

### Translations

In [48]:
def translate(img, direction, shift, roll=True):
  assert direction in ['right', 'left', 'down', 'up'], 'Directions should be top|up|left|right'
  img = img.copy()
  if direction == 'right':
      right_slice = img[:, -shift:].copy()
      img[:, shift:] = img[:, :-shift]
      if roll:
          img[:,:shift] = np.fliplr(right_slice)
  if direction == 'left':
      left_slice = img[:, :shift].copy()
      img[:, :-shift] = img[:, shift:]
      if roll:
          img[:, -shift:] = left_slice
  if direction == 'down':
      down_slice = img[-shift:, :].copy()
      img[shift:, :] = img[:-shift,:]
      if roll:
          img[:shift, :] = down_slice
  if direction == 'up':
      upper_slice = img[:shift, :].copy()
      img[:-shift, :] = img[shift:, :]
      if roll:
          img[-shift:,:] = upper_slice   
  return img    

In [49]:
shift = 10
translated_images = []
directions = ['right', 'left', 'down', 'up']
skip_factor = 1
i = 1
for direction in directions:
  for img in images:
    if i % skip_factor == 0:
      translated_images.append(translate(img,direction,shift))
    i = i + 1  

In [50]:
translated_images = np.asarray(translated_images) 
#translated_images.shape
if images.shape[0] +translated_images.shape[0] < image_limit:
  images = np.concatenate( (images, translated_images) )  

In [51]:
images.shape

(5, 256, 256, 3)

In [52]:
def random_crop(img, crop_size=(10, 10)):
    assert crop_size[0] <= img.shape[0] and crop_size[1] <= img.shape[1], "Crop size should be less than image size"
    img = img.copy()
    w, h = img.shape[:2]
    x, y = np.random.randint(h-crop_size[0]), np.random.randint(w-crop_size[1])
    img = img[y:y+crop_size[0], x:x+crop_size[1]]
    return img

In [53]:
import cv2
def res_image(image):
  image = image.copy()
  resize_image = cv2.resize(image, (256, 256),interpolation=cv2.INTER_NEAREST)
  return resize_image

In [54]:
cropped_images = []
n_crops = 1 #number of crops per image
skip_factor = 1
i = 1
for _ in range (0,n_crops):
  for img in images:
    if i % skip_factor == 0:
      crop_size = (int(img.shape[0]//4),int(img.shape[1]//4)) 
      cropped_images.append(res_image(random_crop(img,crop_size)))
    i = i + 1  

In [55]:
cropped_images = np.asarray(cropped_images) 
if images.shape[0] + cropped_images.shape[0] < image_limit:
  images = np.concatenate( (images, cropped_images) )

In [56]:
images.shape[0]

10

### Rotations

In [57]:
def rotate_img(img, angle, bg_patch=(5,5)):
    assert len(img.shape) <= 3, "Incorrect image shape"
    rgb = len(img.shape) == 3
    if rgb:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1], :], axis=(0,1))
    else:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1]])
    img = rotate(img, angle, reshape=False)
    mask = [img <= 0, np.any(img <= 0, axis=-1)][rgb]
    img[mask] = bg_color
    return img

In [58]:
rotated_images = []
n_rotations = 1 #number of rotated pictures per picture
skip_factor = 1
i = 1
for _ in range (0,n_rotations):
  for img in images:
    if i % skip_factor == 0:
      angle = np.random.randint(359)
      rotated_images.append(rotate_img(img,angle))
    i = i + 1

In [59]:
rotated_images = np.asarray(rotated_images)
if images.shape[0] + rotated_images.shape[0] < image_limit:
  images = np.concatenate((images, rotated_images))
images.shape

(20, 256, 256, 3)

### Random Noise

In [60]:
def gaussian_noise(img, mean=0, sigma=0.03):
    img = img.copy()
    noise = np.random.normal(mean, sigma, img.shape)
    mask_overflow_upper = img+noise >= 1.0
    mask_overflow_lower = img+noise < 0
    noise[mask_overflow_upper] = 1.0
    noise[mask_overflow_lower] = 0
    img = img + noise
    return img

In [61]:
noised_images = []
n_noises = 1
skip_factor = 1 #due to high memory usage it's adviced to skip some images 
i = 1
for _ in range (0,n_noises):
  for img in images:
    if i % skip_factor == 0:
      sigma = round((np.random.randint(10)/121),2)
      noised_images.append(gaussian_noise(img,sigma))

    i = i + 1

In [62]:
noised_images = np.asarray(noised_images)
if images.shape[0] + noised_images.shape[0] < image_limit:
  images = np.concatenate( (images, noised_images) )
images.shape

(40, 256, 256, 3)

### Distortions

In [63]:
def distort(img, orientation='horizontal', func=np.sin, x_scale=0.05, y_scale=5):
    assert orientation[:3] in ['hor', 'ver'], "dist_orient should be 'horizontal'|'vertical'"
    assert func in [np.sin, np.cos], "supported functions are np.sin and np.cos"
    assert 0.00 <= x_scale <= 0.1, "x_scale should be in [0.0, 0.1]"
    assert 0 <= y_scale <= min(img.shape[0], img.shape[1]), "y_scale should be less then image size"
    img_dist = img.copy()
    rgb = len(img.shape) == 3
    def shift(x):
        return int((y_scale * func(np.pi * x * x_scale)))
    
    if rgb:
      for c in range(3):
        for i in range(img.shape[orientation.startswith('ver')]):
            if orientation.startswith('ver'):
                img_dist[:, i, c] = np.roll(img[:, i, c], shift(i))   
            else:
                img_dist[i, :, c] = np.roll(img[i, :, c], shift(i))
    else:
         for i in range(img.shape[orientation.startswith('ver')]):
            if orientation.startswith('ver'):
                img_dist[:, i] = np.roll(img[:, i], shift(i))   
            else:
                img_dist[i, :] = np.roll(img[i, :], shift(i))        
    return img_dist

In [64]:
imgs_distorted = []
skip_factor = 2
i = 1

for img in images:
  for ori in ['ver', 'hor']:
    if i % skip_factor == 0:
      x_param = np.random.randint(1)/10
      y_param = np.random.randint(min(img.shape[0], img.shape[1]) - np.random.randint(min(img.shape[0], img.shape[1])))
      imgs_distorted.append(distort(img, orientation=ori, x_scale=x_param, y_scale=y_param))
    i = i + 1 
imgs_distorted = np.asarray(imgs_distorted)
if images.shape[0] + imgs_distorted.shape[0] < image_limit:
  images = np.concatenate((images, imgs_distorted))

### Color channels change

In [65]:
def change_channel_ratio(img, channel, ratio=0.5):
    assert channel in 'rgb', "Value for channel: rg|b|"
    img = img.copy()
    ci = 'rgb'.index(channel)
    img[:, :, ci] =  img[:, :, ci]*  ratio
    return img

In [66]:
colored_images = []
n_colors = 1
skip_factor = 2 #due to high memory usage it's adviced to skip some images 
i = 1
rgb = len(img.shape) == 3
if rgb:
  for _ in range (0,n_colors):
    for img in images:
      if i % skip_factor == 0:
        colors = ['r','g','b']
        for color in colors:
          sigma = round((np.random.randint(10)/135),2)
          ratio = np.random.randint(10)/14
          colored_images.append(change_channel_ratio(img,color,ratio))
      i = i + 1

In [67]:
if rgb:
  colored_images = np.asarray(colored_images)
  if images.shape[0] + colored_images.shape[0] < image_limit:
    images = np.concatenate((images, colored_images))
images.shape

(200, 256, 256, 3)

In [68]:
def change_channel_ratio_gauss(img, channel='r', mean = 0, sigma=0.03):
    assert channel in 'rgb', "cahenel must be r|g|b"
    img = img.copy()
    ci = 'rgb'.index(channel)
    img[:, :, ci] = gaussian_noise(img[:, :, ci], mean=mean, sigma=sigma)
    return img

In [69]:
colored_images = []
n_colors = 1
skip_factor = 1 #due to high memory usage it's adviced to skip some images 

i = 1
rgb = len(img.shape) == 3
if rgb:
  for _ in range (0,n_colors):
    for img in images:
      if i % skip_factor == 0:
        colors = ['r','g','b']
        for color in colors:
          mean = 0
          sigma = np.random.randint(10)/40
          colored_images.append(change_channel_ratio_gauss(img,color,0,sigma))
      i = i + 1

In [70]:
if rgb:
  colored_images = np.asarray(colored_images)
  if images.shape[0] + colored_images.shape[0] < image_limit:
    images = np.concatenate((images, colored_images))
images.shape

(800, 256, 256, 3)

In [72]:
import os
save_path = destination_path
i = 0
for i in range(0,len(images)-1):
  gen_img = Image.fromarray(images[i].astype(np.uint8))
  file = 'grain_'+str(i+1)
  gen_img.save(save_path+file+'.jpg')
  i = i + 1 